In [1]:
import pandas as pd
import numpy as np
from utils import *
from math import sqrt

c:\Users\azhe1\AppData\Local\Programs\Python\Python311\Lib\site-packages\py_lets_be_rational\numba_helper.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  return jit(*jit_args, **jit_kwargs)(fun)


In [2]:
sp500 = pd.read_csv("data/sp.csv", index_col=0)
sp500.index = pd.to_datetime(sp500.index)

sp500["return"] = (sp500["Adj Close"].shift(periods=-1)/sp500["Adj Close"]).apply(np.log)
sp500["return_6m"] = (sp500["Adj Close"].shift(periods=-21*6)/sp500["Adj Close"]).apply(np.log)
sp500["vol_20d"] = sp500["return"].rolling(window=20).std()*sqrt(252)
sp500["vol_30d"] = sp500["return"].rolling(window=30).std()*sqrt(252)
sp500["vol_1y"] = sp500["return"].rolling(window=252).std()*sqrt(252)
sp500["vol_3y"] = sp500["return"].rolling(window=756).std()*sqrt(252)
sp500 = sp500.dropna()
sp500.head()

,Open,High,Low,Close,Adj Close,Volume,return,return_6m,vol_20d,vol_30d,vol_1y,vol_3y
Date,,,,,,,,,,,,
2003-01-07,929.010010,930.809998,919.929993,922.929993,922.929993,1545200000,-0.014186,0.082409,0.215741,0.221798,0.262761,0.233770
2003-01-08,922.929993,922.929993,908.320007,909.929993,909.929993,1467600000,0.019201,0.083023,0.220739,0.229086,0.263505,0.232989
2003-01-09,909.929993,928.309998,909.929993,927.570007,927.570007,1560300000,0.000000,0.073325,0.220772,0.220268,0.263498,0.232985
2003-01-10,927.580017,932.890015,917.659973,927.570007,927.570007,1485400000,-0.001413,0.079040,0.220230,0.204773,0.263356,0.232984
2003-01-13,927.570007,935.049988,922.049988,926.260010,926.260010,1396300000,0.005813,0.077020,0.213514,0.205442,0.263382,0.232479


In [3]:
r = pd.read_csv("data/risk_free.csv", index_col=0)
r.index = pd.to_datetime(r.index)
r["DTB3"] = r["DTB3"].replace(".", np.nan)
r["DTB3"] = r["DTB3"].ffill()
r["DTB3"] = pd.to_numeric(r["DTB3"])
r.head()

,DTB3
DATE,
1954-01-04,1.33
1954-01-05,1.28
1954-01-06,1.28
1954-01-07,1.31
1954-01-08,1.31


In [4]:
sp500 = pd.merge(sp500, r, how='left', left_index=True, right_index=True)
sp500.rename(columns={"DTB3":"risk_free_rate"}, inplace=True)
sp500.head()

,Open,High,Low,Close,Adj Close,Volume,return,return_6m,vol_20d,vol_30d,vol_1y,vol_3y,risk_free_rate
Date,,,,,,,,,,,,,
2003-01-07,929.010010,930.809998,919.929993,922.929993,922.929993,1545200000,-0.014186,0.082409,0.215741,0.221798,0.262761,0.233770,1.17
2003-01-08,922.929993,922.929993,908.320007,909.929993,909.929993,1467600000,0.019201,0.083023,0.220739,0.229086,0.263505,0.232989,1.17
2003-01-09,909.929993,928.309998,909.929993,927.570007,927.570007,1560300000,0.000000,0.073325,0.220772,0.220268,0.263498,0.232985,1.18
2003-01-10,927.580017,932.890015,917.659973,927.570007,927.570007,1485400000,-0.001413,0.079040,0.220230,0.204773,0.263356,0.232984,1.18
2003-01-13,927.570007,935.049988,922.049988,926.260010,926.260010,1396300000,0.005813,0.077020,0.213514,0.205442,0.263382,0.232479,1.19


In [5]:
sp6m = pd.read_csv("data/sp6m_6.csv", index_col=0)
sp6m.rename(columns={"idt":"Date"}, inplace=True)
sp6m["Date"] = pd.to_datetime(sp6m["Date"])
sp6m.set_index("Date", inplace=True)
sp6m.head()

,market,maturity_target,mu,sd,skew,kurt,p10,p50,p90,lg_change_decr,prDec,lg_change_incr,prInc
Date,,,,,,,,,,,,,
2007-01-12,sp6m,6.0,0.018803,0.086435,-1.079705,1.837194,-0.095498,0.03382,0.111149,-20.0,0.024242,20.0,0.001915
2007-01-31,sp6m,6.0,0.018350,0.077969,-1.046439,1.842209,-0.083769,0.03099,0.102377,-20.0,0.016758,20.0,0.000767
2007-02-15,sp6m,6.0,0.019976,0.084543,-1.001081,1.588657,-0.092096,0.03361,0.111834,-20.0,0.021096,20.0,0.001900
2007-02-28,sp6m,6.0,0.018823,0.100147,-1.031801,1.638698,-0.114332,0.03565,0.126683,-20.0,0.036788,20.0,0.007350
2007-03-15,sp6m,6.0,0.018767,0.099519,-1.058040,1.823271,-0.112902,0.03536,0.125505,-20.0,0.036138,20.0,0.007715


In [10]:
RND_count = 0
GBM_count = 0
total = 0
error = 0

for i in sp6m.index:
    rnp = sp6m.loc[i]
    mean = rnp["mu"]
    std = rnp["sd"]
    skew = rnp["skew"]
    kurt = rnp["kurt"]

    try:
        # RND simulation
        coeff = fit_fleishman_from_sk(skew, kurt)
        sim = (generate_fleishman(-coeff[1],*coeff,N=100000)) * std + mean
        # GBM simulation
        sp = sp500.loc[i]
        s0 = sp["Close"]
        alpha = sp["risk_free_rate"]
        sigma = sp["vol_20d"]
        true_retun = sp["return_6m"]
        T = 1/2
        Nsim = 100000
        stratified = True
        gbm = SimuStockPath(s0, alpha, sigma, T, Nsim, stratified)
        gbm = np.log(gbm/s0)
        # Set consition
        condition = lambda x : x < true_retun + 0.0001 and x > true_retun - 0.0001
        count_gbm = sum(1 for element in gbm if condition(element))
        count_rnp = sum(1 for element in sim if condition(element))

        total += 1
        if count_gbm > count_rnp:
           GBM_count += 1
        elif count_gbm < count_rnp:
           RND_count += 1

    except:
        print(f"Error at {i}")
        error += 1

print(GBM_count/total)
print(RND_count/total)
print(error)
print(total)


Error at 2018-11-22 00:00:00
Error at 2020-03-18 00:00:00
Error at 2022-07-06 00:00:00
Error at 2022-07-13 00:00:00
Error at 2022-07-20 00:00:00
Error at 2022-07-27 00:00:00
Error at 2022-08-03 00:00:00
Error at 2022-08-10 00:00:00
Error at 2022-08-17 00:00:00
Error at 2022-08-24 00:00:00
Error at 2022-08-31 00:00:00
Error at 2022-09-07 00:00:00
Error at 2022-09-14 00:00:00
Error at 2022-09-21 00:00:00
Error at 2022-09-28 00:00:00
Error at 2022-10-05 00:00:00
Error at 2022-10-12 00:00:00
Error at 2022-10-19 00:00:00
Error at 2022-10-26 00:00:00
Error at 2022-11-02 00:00:00
Error at 2022-11-09 00:00:00
Error at 2022-11-16 00:00:00
Error at 2022-11-23 00:00:00
Error at 2022-11-30 00:00:00
Error at 2022-12-07 00:00:00
Error at 2022-12-14 00:00:00
Error at 2022-12-21 00:00:00
Error at 2022-12-28 00:00:00
Error at 2023-01-04 00:00:00
Error at 2023-01-11 00:00:00
Error at 2023-01-18 00:00:00
Error at 2023-01-25 00:00:00
Error at 2023-02-01 00:00:00
Error at 2023-02-08 00:00:00
Error at 2023-